pdf to text : https://wooiljeong.github.io/python/pdf-to-text/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, sys

my_path = '/content/package'
save_path = '/content/drive/MyDrive/Colab Notebooks/package'

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)

In [ ]:
# transformers, chromadb 버전 확인
!pip update --target=$my_path transformers==4.31.0
!pip update --target=$my_path chromadb==0.4.6
!pip install --target=$my_path -U sentence-transformers

ERROR: unknown command "update"
ERROR: unknown command "update"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
^C


In [ ]:
!pip install chromadb==0.4.6

## 허깅페이스에서 임베딩 모델 불러오기

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
model_checkpoint = "jhgan/ko-sroberta-multitask"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [ ]:
print(model)
print(tokenizer)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

**tokenizer**는 학습된 모델에 자연어를 넣고자 할 떄 문장이 있을 시 단어 단위로 쪼개주는 것이다.

임베딩과 다르게 **사전**을 기반하여
입력값에 인덱스를 부여하여 숫자값으로 변환한다.



In [ ]:
# 32000개의 쪼개진 단어를 사전으로 구축하고 있음
tokenizer.vocab_size

32000

`input_ids`: 사전에서의 인덱스값


`cls` : 0이며 문장의 시작

In [ ]:
tokenizer("안녕하세요.")

{'input_ids': [0, 5891, 2205, 5971, 18, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

## 임베딩 생성 / BERT 등의 일반적인 언어 모델일 경우

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def model_encode(text):
    # Tokenize sentences
    encoded_input = tokenizer([text], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    return mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
def embedding_for_text(text):
    return model_encode(text).detach().numpy()[0].tolist()

## 임베딩 생성 / 문장 유사도 모델일 경우

In [ ]:
def embedding_for_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=False)
    outputs = model(**inputs)
    return outputs.last_hidden_state[0].detach().numpy()[0].tolist()

문장 유사도 모델을 통해 아래 문장의 임베딩을 추출한 것이다.

In [ ]:
embedding_for_text("안녕하세요. Growth Hub for Developers, 『튜링의 사과』 입니다.")

[-0.3691728413105011,
 -0.4095826745033264,
 -0.08411337435245514,
 0.6890881657600403,
 -0.1743984967470169,
 -0.5226800441741943,
 0.5122963190078735,
 0.31558525562286377,
 0.21077074110507965,
 0.6156884431838989,
 -0.0830177441239357,
 -0.05244724825024605,
 -0.658449649810791,
 0.7429108619689941,
 -0.16768066585063934,
 0.6004887223243713,
 -0.03540359437465668,
 -0.3141450881958008,
 -0.2840249836444855,
 -0.5399563312530518,
 -0.37883055210113525,
 -0.041289664804935455,
 0.20785808563232422,
 0.3214450478553772,
 -0.2786865234375,
 0.3308762013912201,
 -0.6378315091133118,
 -0.04690467566251755,
 0.18845060467720032,
 -0.05535484850406647,
 -0.15576906502246857,
 0.06674794107675552,
 -0.5379538536071777,
 -0.22861096262931824,
 0.07885751873254776,
 0.4869573414325714,
 0.05614107474684715,
 -0.3879896402359009,
 0.5178214311599731,
 -0.8599975109100342,
 0.22715900838375092,
 -0.35157737135887146,
 0.13448524475097656,
 0.37465137243270874,
 -0.05404165759682655,
 -0.268852

## 코사인 유사도 분석

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
  return dot(a, b)/(norm(a)*norm(b))

In [ ]:
cosine_similarity(
    embedding_for_text("나는 어제 밥을 먹지 않았다"),
    embedding_for_text("나는 전날 식사를 못했다")
)

0.821084078440311

In [ ]:
cosine_similarity(
    embedding_for_text("내가 사랑하는 사람은 눈이 크다"),
    embedding_for_text("나는 귀가 큰 사람을 좋아한다")
)

0.611016706633669

In [ ]:
cosine_similarity(
    embedding_for_text("어제 시험을 보지 못했다"),
    embedding_for_text("전날에 시험이 취소되었다")
)

0.6580797538251558

## RAG로 증강할 지식 문서 처리하기

In [ ]:
document = """
   ===<지진>===
지진이 발생하면 이렇게 대피합니다.
1. 지진으로 흔들릴 때는?
∙ 지진으로 흔들리는 동안은 탁자 아래로 들어가 몸을 보호하고, 탁자 다리를 꼭 잡습니다.
2. 흔들림이 멈췄을 때는?
∙ 흔들림이 멈추면 전기와 가스를 차단하고,문을 열어 출구를 확보합니다.
3. 건물 밖으로 나갈 때는?
∙ 건물 밖으로 나갈 때에는 계단을 이용하여 신속하게 이동합니다. (엘리베이터 사용 금지)
※ 엘리베이터 안에 있을 경우에는 모든 층의 버튼을 눌러 먼저 열리는 층에서 내립니다.
4. 건물 밖으로 나왔을 때는?
∙ 건물 밖에서는 가방이나 손으로 머리를 보호하며, 건물과 거리를 두고 주위를 살피며 대피합니다.
5. 대피 장소를 찾을 때는?
∙ 떨어지는 물건에 유의하며 신속하게 운동장이나 공원 등 넓은 공간으로 대피합니다.(차량 이용 금지)
6. 대피 장소에 도착한 후에는?
∙ 라디오나 공공기관의 안내 방송 등 올바른 정보에 따라 행동합니다.

지진 발생 시 장소에 따라 이렇게 행동합니다.
1. 집안에 있을 경우
∙ 탁자 아래로 들어가 몸을 보호합니다.
∙ 흔들림이 멈추면 전기와 가스를 차단하고 문을 열어 출구를 확보한 후, 밖으로 나갑니다.
2. 집밖에 있을 경우
∙ 떨어지는 물건에 대비하여 가방이나 손으로 머리를 보호하며, 건물과 거리를 두고 운동장이나 공원 등 넓은 공간으로 대피합니다.
3. 엘리베이터에 있을 경우
∙ 모든 층의 버튼을 눌러 가장 먼저 열리는 층에서 내린 후 계단을 이용합니다.
※ 지진 시 엘리베이터를 타면 안됩니다.
4. 학교에 있을 경우
∙ 책상 아래로 들어가 책상 다리를 꼭 잡습니다.
∙ 흔들림이 멈추면 질서를 지키며 운동장으로 대피합니다.
5. 백화점, 마트에 있을 경우
∙ 진열장에서 떨어지는 물건으로부터 몸을 보호하고, 계단이나 기둥 근처로 가 있습니다.
∙ 흔들림이 멈추면 밖으로 대피합니다.
6. 극장‧경기장에 있을 경우
∙ 흔들림이 멈출 때까지 가방 등 소지품으로 몸을 보호하면서 자리에 있다가, 안내에 따라 침착하게 대피합니다.
7. 전철을 타고 있을 경우
∙ 손잡이나 기둥을 잡아 넘어지지 않도록 합니다.
∙ 전철이 멈추면 안내에 따라 행동합니다.
8. 운전을 하고 있는 경우
∙ 비상등을 켜고 서서히 속도를 줄여 도로 오른쪽에 차를 세우고,
라디오의 정보를 잘 들으면서 키를 꽂아 두고 대피합니다.
9. 산이나 바다에 있을 경우
∙ 산사태, 절벽 붕괴에 주의하고 안전한 곳으로 대피합니다.
∙ 해안에서 지진해일 특보가 발령되면 높은 곳으로 이동합니다.

몸이 불편하신 분은 이렇게 행동합니다.
1. 시력이 좋지 않거나 시각장애가 있는 경우
∙ 우선은 라디오나 텔레비전 등으로 상황 파악을 합니다.
∙ 대피가 필요한 경우, 바닥에 낙하물이 있는지 장애물을 점검하며 천천히 움직이고 주위 사람에게 도움을 적극적으로 요청합니다.
∙ 2차 피해를 예방하기 위해 주변 사람에게 화기가 있는 장소의 점검 등을 요청합니다.
2. 거동이 불편하거나 지체장애가 있는 경우
∙ 혼자서 행동하지 말고 이웃과 함께 대피합니다.
∙ 휠체어나 보행기를 사용하는 경우에는 바퀴를 잠그고 몸을 앞으로 숙이고, 책 ‧방석‧베개 등으로 머리와 목을 보호합니다.
∙ 움직일 수 없을 때는 조금이라도 안전한 장소에서 도움을 기다립니다.
∙ 야외 넓은 장소 또는 대피장소에 가지 않고 자택에 머무르는 경우에는 이웃이나 관공서 직원 등에게 그 사실을 알리고 고립되지 않도록 합니다.
3. 청력이 좋지 않거나 청각장애가 있는 경우
∙ 텔레비전의 자막방송과 휴대전화 등으로 정보를 수집하도록 노력합니다.
∙ 움직일 수 없게 되었을 경우에는 호루라기 등으로 소리를 내어 장소를 알리고 도움을 받습니다.
∙ 주변사람들에게 청각장애가 있다는 것을 알리고, 안전한 장소로 유도 및 필요한 배려를 받습니다.
※ 언어장애가 있는 경우에는 의사소통 도구(종이, 펜 등)을 이용하여 상황을 파악합니다.
4. 정신이 불안정하거나 발달장애가 있는 경우
∙ 서둘러서 급히 뛰어가거나 바깥으로 뛰어나가지 않습니다.
∙ 가족이나 도움을 주는 사람들과 미리 정한 것을 지킵니다.
∙ 혼란스러워 스스로 결정하지 못할 때에는 주위사람에게 도움을 요청합니다.

어린이와 함께 있을 때에는 이렇게 행동합니다.
1. 유모차 보다는 아기띠를 사용합니다.
∙ 영유아의 경우 대피 시 유모차 사용이 어렵기 때문에 평소에 업는 띠를 준비해 둡니다. 대피 시 아기는 띠를 이용해 안고 손을 자유롭게 해 아기와 자신의 신체를 보호합니다.
2. 신발을 신겨서 안고 대피합니다.
∙ 지진 상황에서는 유리 파편, 건물 잔해 등으로 도로가 위험하기 때문에 걸을 수 있는 아이라도 안고 대피해야 합니다. 업을 때에는 머리를 보호하고, 안거나 업더라도 반드시 신발을 신깁니다.
3. 손을 꼭 잡고 행동요령을 확인합니다.
∙ 혼란스러운 상황 속에서 헤어지지 않도록 손을 잡고 대피합니다. 어린이의 경우 행동요령을 알고 있더라도 필요한 행동을 말해주며 함께 대처할 수 있도록 안내합니다.
"""

In [ ]:
def split_documents(document, max_length):
    splitted = []
    chunk, doc_length = "", 0
    for doc in [ doc.strip() for doc in document.split("\n") ]:
        if len(doc) == 0:
          continue

        if doc_length > 0 and len(doc) + doc_length > max_length:
            splitted.append(chunk.strip())
            chunk = ""
            doc_length = 0
        chunk += doc + " "
        doc_length += len(doc)

    if len(chunk) > 0:
        splitted.append(chunk)

    return splitted

`split_documents`는 문장의 입력에 한계가 있기 때문에 사용하는 함수이다.

768개의 token을 받을 수 있는 모델을 사용하고 있는데 문장이 몇 개가 될지는 tokenizer로 쪼개봐야 알 수 있는 것이어서 768개보다 적은 128개 단위로 문장을 쪼개도록 한다.

In [ ]:
documents = split_documents(document, 768)

In [ ]:
documents[:3]

['===<지진>=== 지진이 발생하면 이렇게 대피합니다. 1. 지진으로 흔들릴 때는? ∙ 지진으로 흔들리는 동안은 탁자 아래로 들어가 몸을 보호하고, 탁자 다리를 꼭 잡습니다. 2. 흔들림이 멈췄을 때는? ∙ 흔들림이 멈추면 전기와 가스를 차단하고,문을 열어 출구를 확보합니다. 3. 건물 밖으로 나갈 때는? ∙ 건물 밖으로 나갈 때에는 계단을 이용하여 신속하게 이동합니다. (엘리베이터 사용 금지) ※ 엘리베이터 안에 있을 경우에는 모든 층의 버튼을 눌러 먼저 열리는 층에서 내립니다. 4. 건물 밖으로 나왔을 때는? ∙ 건물 밖에서는 가방이나 손으로 머리를 보호하며, 건물과 거리를 두고 주위를 살피며 대피합니다. 5. 대피 장소를 찾을 때는? ∙ 떨어지는 물건에 유의하며 신속하게 운동장이나 공원 등 넓은 공간으로 대피합니다.(차량 이용 금지) 6. 대피 장소에 도착한 후에는? ∙ 라디오나 공공기관의 안내 방송 등 올바른 정보에 따라 행동합니다. 지진 발생 시 장소에 따라 이렇게 행동합니다. 1. 집안에 있을 경우 ∙ 탁자 아래로 들어가 몸을 보호합니다. ∙ 흔들림이 멈추면 전기와 가스를 차단하고 문을 열어 출구를 확보한 후, 밖으로 나갑니다. 2. 집밖에 있을 경우 ∙ 떨어지는 물건에 대비하여 가방이나 손으로 머리를 보호하며, 건물과 거리를 두고 운동장이나 공원 등 넓은 공간으로 대피합니다. 3. 엘리베이터에 있을 경우 ∙ 모든 층의 버튼을 눌러 가장 먼저 열리는 층에서 내린 후 계단을 이용합니다. ※ 지진 시 엘리베이터를 타면 안됩니다. 4. 학교에 있을 경우',
 '∙ 책상 아래로 들어가 책상 다리를 꼭 잡습니다. ∙ 흔들림이 멈추면 질서를 지키며 운동장으로 대피합니다. 5. 백화점, 마트에 있을 경우 ∙ 진열장에서 떨어지는 물건으로부터 몸을 보호하고, 계단이나 기둥 근처로 가 있습니다. ∙ 흔들림이 멈추면 밖으로 대피합니다. 6. 극장‧경기장에 있을 경우 ∙ 흔들림이 멈출 때까지 가방 등 소지품으로 몸을 보호하면서 자리에 있다가, 안내에 따라 침착하게 대

## 벡터 데이터베이스 생성

In [ ]:
!pip install --target=$my_path chromadb==0.4.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
import chromadb

In [ ]:
# client = chromadb.PersistentClient(path="./turingapple.db")

In [ ]:
client = chromadb.PersistentClient(path="./myDB.db")

In [ ]:
def embedding_function(texts):
    return [ embedding_for_text(text) for text in texts]

In [ ]:
db = client.get_or_create_collection(
    name="document",
    embedding_function=embedding_function
)
print(db)

In [ ]:
for idx, doc in enumerate(documents):
    db.add(
        documents=[ doc ],
        ids=[ str(idx) ]
    )

## 벡터 데이터베이스 질의를 통한 문맥 및 프롬프트 생성용 함수들

In [ ]:
def build_context(question):
    context = ""
    for doc in db.query(query_embeddings=embedding_for_text(question), n_results=7)["documents"][0]:
        context += doc + '\n\n'
    return context

In [ ]:
def generate_prompt(question, context):
    return f"다음 문맥에 기반해서 질문에 대해 대처법을 알려줘.\n\n문맥:\n{context}\n질문: {question}"

## 실제 질의에 기반한 프롬프트 생성하기

In [ ]:
question = "[기상청] 11월 30일 04:55 경북 경주시 동남동쪽 19km 지역 규모4.3 지진발생/낙하물 주의, 국민재난안전포털 행동요령에 따라 대응, 여진주의."
context = build_context(question)
prompt = generate_prompt(question, context)
print(prompt)

In [ ]:
question = "현재 지하철에 있어."
context = build_context(question)
prompt = generate_prompt(question, context)
print(prompt)